In [1]:
# import API KEY

import os                           # operating system library
from dotenv import load_dotenv      # load environment variables  


load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai.chat_models import ChatOpenAI   # LangChain connection to OpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4-turbo")

response = model.invoke("What is the Suez Canal?")

response.content

"The Suez Canal is a man-made waterway in Egypt, connecting the Mediterranean Sea to the Red Sea. This canal is a crucial global shipping route because it allows vessels to travel between Europe and South Asia without navigating around Africa, thereby significantly reducing travel time and distance.\n\nOpened in 1869, the canal was engineered by the French diplomat Ferdinand de Lesseps. It is about 193 kilometers (120 miles) long, 24 meters (79 feet) deep, and, after several expansions, up to 205 meters (673 feet) wide. The Suez Canal does not contain locks because the water level in both the Mediterranean and the Red Sea are roughly the same, allowing ships to sail smoothly between the two seas.\n\nThe canal is strategically and economically significant. It handles about 10% of the world's maritime trade, including the transportation of oil and natural gas, making it a key asset in international trade. The canal has also been the focus of geopolitical interest and conflicts, notably d

In [3]:
os.listdir("../pdfs")

['.DS_Store', '2024_BRIEFING.pdf']

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [5]:
loader = PyPDFDirectoryLoader("../pdfs/")

pages = loader.load()

In [6]:
len(pages)

8

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings


vectorizer = OpenAIEmbeddings()

In [8]:
from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(pages, vectorizer, persist_directory="../chroma_db")

In [9]:
retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "lambda_mult": 0.25})

In [10]:
from langchain.prompts import ChatPromptTemplate

In [11]:
template = """
            Given the context below and the question, 
            please generate a header, and 5 bullet points, Summarize each bullet point in 40 words.
            Also fetch sub_titles and numbers to describe the information.

            Context: {context}

            Question: {question}
            """


prompt = ChatPromptTemplate.from_template(template)

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
from langchain_core.runnables import RunnablePassthrough

In [14]:
query = "What are the endnotes of the briefing?"

In [15]:
in_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | parser


response = in_chain.invoke(query)

In [16]:
response.split('\n')

['Header: Key Insights from the European Parliament Briefing on Red Sea Security Threats',
 '',
 '1. **Impact of Houthi Attacks on Shipping Routes**',
 '   - Since mid-November 2023, Houthi forces have targeted Western ships near the Bab el-Mandeb Strait, forcing major shipping companies to reroute from the Suez Canal, thereby increasing transit times and disrupting global supply chains.',
 '',
 '2. **Economic Consequences of Shipping Diversions**',
 '   - The diversion of shipping routes to the Cape of Good Hope due to attacks and Panama Canal issues has led to increased shipping durations and costs. This has implications for global trade, potentially affecting energy supplies and contributing to inflation pressures.',
 '',
 '3. **Risks to Submarine Communication Cables**',
 '   - Damage to submarine communications cables, vital for global connectivity, poses significant risks. This infrastructure is crucial for secure government and commercial communications globally.',
 '',
 "4. **E

In [17]:
from langchain_openai import OpenAI

input_model = OpenAI(temperature=0, max_tokens=1024)

In [18]:
template = """
            We have provided  information below.
            Given this information, please generate python-pptx code for a single 
            slide with this information set header and subtitle as title and  
            bullet points.
            
            Put the title on top of the slide and center all text to slide size.
            
            Separate the bullet points into separate texts with line separator.
            Set font size to minimum for fixing text to cells. Save the file in ../pptx folder

            Information: {context}
            """


prompt = ChatPromptTemplate.from_template(template)

In [19]:
out_chain = prompt | input_model | parser

In [20]:
output = out_chain.invoke({"context": response})

In [21]:
output.split('\n')

['',
 '# Import necessary libraries',
 'from pptx import Presentation',
 'from pptx.util import Inches',
 '',
 '# Create a new presentation',
 'prs = Presentation()',
 '',
 '# Add a slide with title and subtitle',
 'slide = prs.slides.add_slide(prs.slide_layouts[0])',
 'title = slide.shapes.title',
 'subtitle = slide.placeholders[1]',
 'title.text = "Key Insights from the European Parliament Briefing on Red Sea Security Threats"',
 'subtitle.text = "Information:"',
 '',
 '# Set text alignment to center',
 'title.text_frame.paragraphs[0].alignment = 1',
 'subtitle.text_frame.paragraphs[0].alignment = 1',
 '',
 '# Add bullet points',
 'bullet_points = [',
 '    "Impact of Houthi Attacks on Shipping Routes",',
 '    "- Since mid-November 2023, Houthi forces have targeted Western ships near the Bab el-Mandeb Strait, forcing major shipping companies to reroute from the Suez Canal, thereby increasing transit times and disrupting global supply chains.",',
 '    "Economic Consequences of Shipp

In [22]:
exec('print(2+2)')

4


In [23]:
exec(output)